In [1]:
import json
import getpass
import requests
import wget
import sys
from usgs import api

# get api key

In [2]:
#Form JSON Object to store credentials to generate API Key.
username='rbavery'
password = getpass.getpass()

login = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login'

creds={"username": username,
"password": password,
"authType": "",
"catalogId": "EE"}

json_data = {'jsonRequest' : json.dumps(creds)}

r = requests.post(login, data = json_data)
token = r.json()['data']

if token == None:
    print("API Key Failed to Generate")

else:
    print("Your API Key Generated Successfully")
    print(token)
#If Key fails to generate please see documentmentation to decode error https://earthexplorer.usgs.gov/inventory/documentation/errors

 ·········


Your API Key Generated Successfully
81d6f34896ab41faaae307af3995e53c


# make a geojson feature

In [3]:
feature_coll = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -104.1064453125,
              43.13306116240612
            ],
            [
              -104.12841796875,
              41.04621681452063
            ],
            [
              -102.19482421875,
              41.0130657870063
            ],
            [
              -102.15087890624999,
              40.027614437486655
            ],
            [
              -95.07568359375,
              40.01078714046552
            ],
            [
              -95.625,
              40.730608477796636
            ],
            [
              -96.13037109375,
              42.17968819665961
            ],
            [
              -97.27294921875,
              43.02071359427862
            ],
            [
              -98.02001953125,
              42.90816007196054
            ],
            [
              -98.54736328125,
              43.08493742707592
            ],
            [
              -104.1064453125,
              43.13306116240612
            ]
          ]
        ]
      }
    }
  ]
}

# make it a geojson object

In [8]:
import geojson
import pprint
import pandas as pd

In [5]:
geojson_aoi = geojson.feature.GeoJSON(feature_coll)

# get dataset fields

In [19]:
def make_date_strings(start_date, end_date):

    return [i.date().isoformat() for i in pd.date_range(start_date, end_date).tolist() if i.month in [6, 7, 8, 9]]

In [31]:
where_or_dict = {
    21791 : ["None", '10'] # this is the field for cloud cover for ARD_TILE
}

where_or_dict_2 = {
    21795 : make_date_strings("2002-06-01", "2019-09-30")
}

In [32]:
where_and_dict = {
     21791 : '100' # this is the field for cloud cover for ARD_TILE
}

In [33]:
def make_child_value_filter(field_value_dict):

    return [{
        "filterType": "value",
        "fieldId": field_id,
        "valueList": value,
        "operand": "="
    } for field_id, value in iter(field_value_dict.items())]

def make_or_filter(child_filter):
    
    return {
        "filterType": "or",
                    "childFilters": child_filter
        }

def make_and_filter(or_filters):
    return {
            "filterType": "and",
            "childFilters": or_filters
        }
    

In [34]:
or1 = make_or_filter(make_child_value_filter(where_or_dict))
or2 = make_or_filter(make_child_value_filter(where_or_dict_2))

In [35]:
filter_str = json.dumps(make_and_filter([or1, or2]))

In [38]:
import ast
filter_json = ast.literal_eval(filter_str)

In [36]:
filter_json = json.loads(filter_str)

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [40]:
filter_json = {"filterType": "and",

 "childFilters": [
   {"filterType": "or",
   "childFilters": [
     {"filterType": "value",
     "fieldId": 21791,
     "valueList": [None, "10"],
     "operand": "="}
    ]
    }
     ]
    } # had to manually change all these to double quoted. WHY?! ast package may solve..

In [18]:
filter_json

{'filterType': 'and',
 'childFilters': [{'filterType': 'or',
   'childFilters': [{'filterType': 'value',
     'fieldId': 21791,
     'valueList': [None, '10'],
     'operand': '='}]}]}

The wrapper of the JSON API can be pip installed from my fork, required for geojson search

`!pip install git+https://github.com/rbavery/usgs`

In [57]:
api.dataset_fields("ARD_TILE", node="EE", api_key=token)

{'errorCode': None,
 'error': '',
 'data': [{'fieldId': 21790,
   'name': 'Tile Identifier',
   'fieldLink': 'https://lta.cr.usgs.gov/DD/ARDTile.html#tile_identifier',
   'valueList': []},
  {'fieldId': 21789,
   'name': 'Tile Grid Region',
   'fieldLink': 'https://lta.cr.usgs.gov/DD/ARDTile.html#tile_grid_region',
   'valueList': [{'value': None, 'name': 'All'},
    {'value': 'CU', 'name': 'CU'},
    {'value': 'AK', 'name': 'AK'},
    {'value': 'HI', 'name': 'HI'}]},
  {'fieldId': 21787,
   'name': 'Tile Grid Horizontal',
   'fieldLink': 'https://lta.cr.usgs.gov/DD/ARDTile.html#tile_grid_horizontal',
   'valueList': []},
  {'fieldId': 21788,
   'name': 'Tile Grid Vertical',
   'fieldLink': 'https://lta.cr.usgs.gov/DD/ARDTile.html#tile_grid_vertical',
   'valueList': []},
  {'fieldId': 21795,
   'name': 'Tile Production Date (Ex. YYYY/MM/DD)',
   'fieldLink': 'https://lta.cr.usgs.gov/DD/ARDTile.html#tile_production_date',
   'valueList': []},
  {'fieldId': 21791,
   'name': 'Cloud Cove

In [47]:
geojson_aoi

{"features": [{"geometry": {"coordinates": [[[-104.1064453125, 43.13306116240612], [-104.12841796875, 41.04621681452063], [-102.19482421875, 41.0130657870063], [-102.15087890624999, 40.027614437486655], [-95.07568359375, 40.01078714046552], [-95.625, 40.730608477796636], [-96.13037109375, 42.17968819665961], [-97.27294921875, 43.02071359427862], [-98.02001953125, 42.90816007196054], [-98.54736328125, 43.08493742707592], [-104.1064453125, 43.13306116240612]]], "type": "Polygon"}, "properties": {}, "type": "Feature"}], "type": "FeatureCollection"}

In [41]:
results = api.search(dataset="ARD_TILE", node="EE", start_date="2005-06-01", end_date="2005-06-30", geojson=geojson_aoi, api_key=token, additional_criteria=filter_json)

In [45]:
import inspect
from usgs import payloads
inspect.getsource(payloads.search)


'def search(dataset, node, geojson=None,\n    lat=None, lng=None,\n    distance=100,\n    ll=None, ur=None,\n    start_date=None, end_date=None,\n    additional_criteria=None,\n    where_and=None,\n    where_or=None,\n    max_results=50000,\n    starting_number=1,\n    sort_order="DESC",\n    api_key=None):\n    """\n    :param dataset:\n    \n    :param node:\n    \n    :param lat:\n    \n    :param lng:\n    \n    :param ll:\n    \n    :param distance:\n    \n    :param ur:\n    \n    :param start_date:\n    \n    :param end_date:\n    \n    :param where_and:\n        Specify additional search criteria with and logic\n\n    :param where_or:\n        Specify additional search criteria with or logic\n    \n    :param max_results:\n    \n    :param starting_number:\n    \n    :param sort_order:\n    \n    :param api_key:\n        API key is not required.\n    """\n\n    payload = {\n        "datasetName": dataset,\n        "node": node,\n        "apiKey": api_key,\n    }\n\n    if geojs

In [42]:
results['data']

{'numberReturned': 5471,
 'totalHits': 5471,
 'firstRecord': 1,
 'lastRecord': 5471,
 'nextRecord': 5471,
 'results': [{'acquisitionDate': '2005-06-30',
   'startTime': '2005-06-30',
   'endTime': '2005-06-30',
   'spatialFootprint': {'type': 'Polygon',
    'coordinates': [[[-162.613982772, 63.9100782532],
      [-159.16546479, 63.9100782532],
      [-159.16546479, 65.3858112934],
      [-162.613982772, 65.3858112934],
      [-162.613982772, 63.9100782532]]]},
   'sceneBounds': '-162.613982772,63.9100782532,-159.16546479,65.3858112934',
   'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_etm_c1/ARD_Tile/2005/AK/003/005/LE07_AK_003005_20050630_C01_V01.jpg',
   'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=ARD_TILE&ordered=LE07_AK_003005_20050630_C01_V01&node=INVSVC',
   'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/ARD_TILE/LE07_AK_003005_20050630_C01_V01/INVSVC/',
   'entityId': 'LE07_AK_003005_20050630_C01_V01',
   'displayI

In [3]:
additional_criteria

{'filterType': 'or',
 'childFilters': [{'filterType': 'value',
   'fieldId': 21791,
   'value': [None, '10', '20'],
   'operand': '='}]}

In [88]:
where_and_dict = {
     21791 : '10' # this is the field for cloud cover for ARD_TILE
}

In [4]:
results = api.search("ARD_TILE", "EE", start_date="2005-06-01", end_date="2005-08-07", geojson=str(geojson_aoi), api_key=token, where=where_or_dict)

NameError: name 'api' is not defined

In [90]:
results['data']

{'numberReturned': 2,
 'totalHits': 2,
 'firstRecord': 1,
 'lastRecord': 2,
 'nextRecord': 2,
 'results': [{'acquisitionDate': '2011-04-04',
   'startTime': '2011-04-04',
   'endTime': '2011-04-04',
   'spatialFootprint': {'type': 'Polygon',
    'coordinates': [[[-123.986221879, 36.2155381302],
      [-121.875673362, 36.2155381302],
      [-121.875673362, 37.8750826792],
      [-123.986221879, 37.8750826792],
      [-123.986221879, 36.2155381302]]]},
   'sceneBounds': '-123.986221879,36.2155381302,-121.875673362,37.8750826792',
   'browseUrl': 'https://ims.cr.usgs.gov/browse/landsat_tm_c1/ARD_Tile/2011/CU/001/009/LT05_CU_001009_20110404_C01_V01.jpg',
   'dataAccessUrl': 'https://earthexplorer.usgs.gov/order/process?dataset_name=ARD_TILE&ordered=LT05_CU_001009_20110404_C01_V01&node=INVSVC',
   'downloadUrl': 'https://earthexplorer.usgs.gov/download/external/options/ARD_TILE/LT05_CU_001009_20110404_C01_V01/INVSVC/',
   'entityId': 'LT05_CU_001009_20110404_C01_V01',
   'displayId': 'LT05_

# There should be more results returned from Nebraska (the geojson used to search) throughout the year, I've confirmed this by downloading ARD manually.

# scratch code below

In [ ]:
where_and_dict = {
     21787 : ["011","012","013","014","015","016","017"], # tile grid horizontal
     21788 : ["006","007","008","009", "010"], # tile grid vertical
     21791 : '10', # cloud cover
     21792 : ['OLI_TIRS', 'TM']
}

additionalCriteria = {
    "filterType": "and",
    "childFilters": [
        {
            "filterType": "value",
            "fieldId": field_id,
            "value": value,
            "operand": "="
        }
        for field_id, value in iter(where.items())
    ]
}

In [ ]:
searchUrl='https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/search'
dataset_search = {"apiKey": (token), "datasetName": "ARD_TILE"}
dataset_request =requests.get(searchUrl, params={"jsonRequest":json.dumps(dataset_search)})
d_resultslist = dataset_request.json()['data']['results']
order = []
for res in d_resultsList:
    order.append(res['entityId'])

#Check to see if query resulted in scenes being returned.
if len(order) == 0:
    sys.exit("No Scenes Matching Criteria Were Found, Program will terminate")
else:
    print("Your Scene List Contains:")
    print(order)

In [22]:
import pandas as pd

In [43]:
def read_dataset_json(fname):
    return pd.DataFrame(pd.read_json(fname).data.apply(lambda x: x['datasetName']))

In [4]:
#Create routine to generate download URLs for desired product for enity ids from search query. Please consult Get Bulk Download
#product endpoint for available products per dataset. Here Level-1 Landsat Geotiff will be downloaded. The product is downloaded
#using wget.

for enityID in [order]:
  orderParms ={'datasetName':dataset,
          'products':'STANDARD',
          'entityIds': order,
          "apiKey": (token)
          
          }
  
  downloadURL="https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/download"
  download= requests.get(downloadURL, params={'jsonRequest':json.dumps(orderParms)})
    
urlList = []
download_list = download.json()['data']
for item in download_list:
    urlList.append(item['url'])

#Users will need to specify a download directory on their machine.
outDir="YOUR DIR"
for URL in urlList:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(order)
print("Download Complete")

The Following Scenes have downloaded:
['LC80260272014047LGN01', 'LC80260272014063LGN01', 'LC80260272014159LGN01', 'LC80260272014207LGN01', 'LC80260272014319LGN01']
Download Complete
